In [1]:
"""from google.colab import drive
drive.mount('/content/drive')"""

Mounted at /content/drive


In [1]:
!pip install pymatreader


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from pymatreader import read_mat
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import os

data_path = '/athena/madelab/scratch/iqh4001/Kinect Project/Dataset/Data/Kinect-ACC-Cleaned'

#data_path = "/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample"

In [2]:



def kinz_time(kinz_data):
    frames_num = len(kinz_data)
    time_in_sec = np.zeros(frames_num)
    time0 = kinz_data[0]  # time zero for the acquisition

    for i in range(frames_num):
        time_dbl = kinz_data[i] - time0  # relative time (double precision)
        time_in_sec[i] = time_dbl / 1e9  # Convert time to seconds.

    mean_time_diff = np.mean(np.diff(time_in_sec))  # Mean time difference
    fs = 1 / mean_time_diff  # Sampling rate

    return time_in_sec, fs




def KINZ_DATA_np(k):

  details = os.path.basename(k)

  p_id= details.split("_")[0].split("-")[-2]+"-"+ details.split("_")[0].split("-")[-1]
  traial = details.split("_")[1]
  walking_speed = details.split("_")[2]

  kinzData = read_mat(k)

  kinz_arr = np.empty([0,100])


  for i in range(len(kinzData["joint_positions"])):
      timeStamps = np.array(kinzData["t_uniform"][i])
      kinz_x = kinzData["joint_positions"][i][0]
      kinz_y = kinzData["joint_positions"][i][1]
      kinz_z = kinzData["joint_positions"][i][2]

      d = np.concatenate((p_id,traial,walking_speed,timeStamps,kinz_x,kinz_y,kinz_z), axis=None)

      kinz_arr = np.append(kinz_arr, [d], axis=0)


  return kinz_arr




In [3]:



final = np.empty([0,100])

for i in tqdm(sorted(glob(data_path+"/*"))):
  for k in sorted(glob(i+"/TS*.mat")):
    print("Processing: ", os.path.basename(k))
    temp_single = KINZ_DATA_np(k)
    #time_elements = temp_single[:, 3]

    #time_elements = time_elements.astype(float)
    #replacement_values = kinz_time(time_elements)[0]

    #temp_single[:, 3] = replacement_values
    final = np.append(final, temp_single, axis=0)





joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]


all_columns = ['patientID','trial_num','walking_speed','timeStamps']


for joint in joints:
  x = f'{joint}_X'
  all_columns.append(x)

for joint in joints:
  y = f'{joint}_Y'
  all_columns.append(y)

for joint in joints:
  z = f'{joint}_Z'
  all_columns.append(z)




final_df = pd.DataFrame(final, columns=all_columns)

columns_to_convert = final_df.columns.difference(['patientID','trial_num','walking_speed','timeStamps'])
final_df[columns_to_convert] = final_df[columns_to_convert].astype(float)




  0%|          | 0/107 [00:00<?, ?it/s]

Processing:  TS-Cln-001-LO_Trial1_Regular_20220307_111113.mat
Processing:  TS-Cln-001-LO_Trial2_Regular_20220307_111139.mat
Processing:  TS-Cln-001-LO_Trial3_Fast_20220307_111208.mat
Processing:  TS-Cln-001-LO_Trial4_Fast_20220307_111234.mat
Processing:  TS-Cln-001-LO_Trial5_Fast_20220307_111347.mat
Processing:  TS-Cln-001-LO_Trial6_Fast_20220307_111427.mat
Processing:  TS-Cln-001-LO_Trial6_Fast_20220307_111722.mat
Processing:  TS-Cln-002-RJ_Trial2_Regular_20220311_100440.mat
Processing:  TS-Cln-002-RJ_Trial3_Fast_20220311_100509.mat
Processing:  TS-Cln-002-RJ_Trial4_Fast_20220311_100541.mat
Processing:  TS-Cln-002-RJ_Trial4_Fast_20220311_100659.mat
Processing:  TS-Cln-RM-003_Trial1_Regular_20220315_060038.mat
Processing:  TS-Cln-RM-003_Trial2_Regular_20220315_060059.mat
Processing:  TS-Cln-RM-003_Trial3_Fast_20220315_060209.mat
Processing:  TS-Cln-RM-003_Trial4_Fast_20220315_060226.mat
Processing:  TS-Cln-RM-003_Trial4_Fast_20220315_060742.mat
Processing:  TS-Cln-004-MF_Trial3_Regular

In [4]:
final_df

,patientID,trial_num,walking_speed,timeStamps,PELVIS_X,SPINE_NAVAL_X,SPINE_CHEST_X,NECK_X,CLAVICLE_LEFT_X,SHOULDER_LEFT_X,...,HIP_RIGHT_Z,KNEE_RIGHT_Z,ANKLE_RIGHT_Z,FOOT_RIGHT_Z,HEAD_Z,NOSE_Z,EYE_LEFT_Z,EAR_LEFT_Z,EYE_RIGHT_Z,EAR_RIGHT_Z
0,001-LO,Trial1,Regular,0.014806249999999993,513.079538,516.003537,527.994788,545.712851,516.264630,405.175317,...,919.948335,929.656427,825.884006,894.117256,925.016797,954.213183,938.401915,886.661529,968.354628,981.439603
1,001-LO,Trial1,Regular,0.03478671874999914,518.341956,523.496677,539.288301,549.834306,520.128180,405.456134,...,934.181465,924.690502,784.109292,847.342565,961.514530,951.963861,963.009239,945.312977,991.238931,1044.996519
2,001-LO,Trial1,Regular,0.054767187499999176,529.923963,541.248914,552.630499,552.997228,524.350174,408.479201,...,980.474516,936.771136,777.246423,837.827719,987.262010,983.921111,995.232664,973.751899,1023.415136,1073.302525
3,001-LO,Trial1,Regular,0.07474765624999921,534.861455,548.038017,556.688923,551.944125,523.861126,407.510892,...,1019.964084,967.044120,802.468369,862.476378,1015.462905,1023.426646,1033.121369,1005.673693,1059.598333,1098.709654
4,001-LO,Trial1,Regular,0.09472812499999925,534.057534,545.354058,552.725394,547.248025,519.301345,403.093700,...,1053.574877,1013.036813,855.414270,916.645154,1045.783754,1069.454529,1075.905348,1040.604981,1099.243905,1121.611912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117155,104-AB,Trial4,Fast,4.061033224999999,-91.218866,-89.747315,-88.313575,-85.556212,-55.779262,54.685006,...,672.652892,696.286269,787.436774,679.529924,701.914488,584.192664,604.260842,666.100653,623.339053,739.626322
117156,104-AB,Trial4,Fast,4.08101369375,-92.125914,-90.266140,-88.525753,-87.422600,-57.603891,52.599955,...,660.061086,676.445955,764.835753,655.871117,678.235995,575.603405,596.319300,646.799074,617.545475,727.644934
117157,104-AB,Trial4,Fast,4.100994162499999,-92.715923,-90.658826,-88.788015,-89.446664,-59.569707,50.333896,...,648.881353,659.209415,745.113326,635.693049,657.047192,568.234716,589.783758,630.401956,612.484297,716.378949
117158,104-AB,Trial4,Fast,4.12097463125,-92.706992,-91.370047,-90.183577,-92.151750,-62.182794,46.564745,...,641.359020,652.310696,734.087816,623.641664,648.228866,565.407634,586.953088,623.371678,610.100149,710.783095


In [16]:

final_df['patientID'].unique().tolist()
#final_df['patientID'] = final_df['patientID'].replace('RM-003', '003-RM')




['001-LO',
 '002-RJ',
 '003-RM',
 '004-MF',
 '005-GP',
 '006-TR',
 '007-LI',
 '008-BJ',
 '009-TB',
 '010-RA',
 '011-CB',
 '012-JM',
 '013-MS',
 '014-WF',
 '015-IL',
 '016-GE',
 '017-CS',
 '018-OS',
 '019-ER',
 '020-MM',
 '021-GA',
 '022-ND',
 '023-PH',
 '024-PO',
 '025-PS',
 '027-GO',
 '028-DF',
 '029-JC',
 '030-RE',
 '032-GB',
 '033-JF',
 '034-MJ',
 '035-LT',
 '037-MA',
 '038-LB',
 '039-EB',
 '040-KR',
 '041-JL',
 '042-PB',
 '044-MS',
 '046-MI',
 '047-MW',
 '048-TT',
 '049-RL',
 '050-TE',
 '051-BV',
 '052-EF',
 '054-LC',
 '055-HK',
 '056-HT',
 '057-JH',
 '058-ZS',
 '059-ES',
 '060-CC',
 '061-JM',
 '063-JS',
 '064-SC',
 '065-JH',
 '067-MT',
 '068-GM',
 '069-CR',
 '070-TC',
 '071-EK',
 '072-DD',
 '073-TP',
 '074-RK',
 '075-RW',
 '076-AH',
 '079-KO',
 '080-TW',
 '081-PB',
 '082-BK',
 '083-PN',
 '084-SL',
 '085-AH',
 '086-CR',
 '087-DO',
 '088-HR',
 '089-JH',
 '090-MF',
 '091-WF',
 '092-CG',
 '093-NO',
 '094-JB',
 '095-MT',
 '096-RF',
 '097-MH',
 '098-CM',
 '099-RC',
 '100-JG',
 '101-WZ',

In [18]:
final_df.columns.tolist()

['patientID',
 'trial_num',
 'walking_speed',
 'timeStamps',
 'PELVIS_X',
 'SPINE_NAVAL_X',
 'SPINE_CHEST_X',
 'NECK_X',
 'CLAVICLE_LEFT_X',
 'SHOULDER_LEFT_X',
 'ELBOW_LEFT_X',
 'WRIST_LEFT_X',
 'HAND_LEFT_X',
 'HANDTIP_LEFT_X',
 'THUMB_LEFT_X',
 'CLAVICLE_RIGHT_X',
 'SHOULDER_RIGHT_X',
 'ELBOW_RIGHT_X',
 'WRIST_RIGHT_X',
 'HAND_RIGHT_X',
 'HANDTIP_RIGHT_X',
 'THUMB_RIGHT_X',
 'HIP_LEFT_X',
 'KNEE_LEFT_X',
 'ANKLE_LEFT_X',
 'FOOT_LEFT_X',
 'HIP_RIGHT_X',
 'KNEE_RIGHT_X',
 'ANKLE_RIGHT_X',
 'FOOT_RIGHT_X',
 'HEAD_X',
 'NOSE_X',
 'EYE_LEFT_X',
 'EAR_LEFT_X',
 'EYE_RIGHT_X',
 'EAR_RIGHT_X',
 'PELVIS_Y',
 'SPINE_NAVAL_Y',
 'SPINE_CHEST_Y',
 'NECK_Y',
 'CLAVICLE_LEFT_Y',
 'SHOULDER_LEFT_Y',
 'ELBOW_LEFT_Y',
 'WRIST_LEFT_Y',
 'HAND_LEFT_Y',
 'HANDTIP_LEFT_Y',
 'THUMB_LEFT_Y',
 'CLAVICLE_RIGHT_Y',
 'SHOULDER_RIGHT_Y',
 'ELBOW_RIGHT_Y',
 'WRIST_RIGHT_Y',
 'HAND_RIGHT_Y',
 'HANDTIP_RIGHT_Y',
 'THUMB_RIGHT_Y',
 'HIP_LEFT_Y',
 'KNEE_LEFT_Y',
 'ANKLE_LEFT_Y',
 'FOOT_LEFT_Y',
 'HIP_RIGHT_Y',
 'KNE

In [17]:
from datetime import datetime

# Generate timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M%p')

# Define save path
saving_path="/athena/madelab/scratch/iqh4001/Kinect Project/Dataset/Data/pre-processed data"
file_name = f"Kinect_data_S0-S104_raw_{timestamp}.csv"

# Save CSV with timestamp
final_df.to_csv(f"{saving_path}/{file_name}", index=False)
